In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [69]:
def get_data(code, name):
    url = f'https://www.pro-football-reference.com/teams/{code}/2024_roster.htm'
    response = requests.get(url)
    html = BeautifulSoup(response.content, 'html.parser')
    h = html.body.find('div', attrs={'data-template': 'Partials/Teams/Summary'}).find_all('p')

    header = {}
    for i in range(len(h[1:])):
        link = h[i].find('a')
        if link:
            header[h[i].find('strong').text[:-1]] = link.text
    header['code'] = code
    header['team'] = name

    t = response.text.split('tbody')
    players = []
    for row in t[2].split('</tr>')[:-1]:
        pnum = BeautifulSoup(row[1:] + '</tr>').find('th').text
        # Player Info
        pinfo = [i.text if i.text else None for i in BeautifulSoup(row[1:] + '</tr>').findAll('td')]

        players.append([pnum, *pinfo[:-2], pinfo[-1]])

    df = pd.DataFrame(players, columns=['num', 'player', 'age', 'pos', 'g', 'gs', 'wt', 'ht', 'college', 'birth', 'yrs', 'draft'])
    df.num = df.num.replace('', pd.NA).astype('Int8')
    df['code'] = code
    df['team'] = name

    return pd.DataFrame(header, index=[0]), df

In [54]:
r2 = requests.get('https://www.pro-football-reference.com/teams/')
h2 = BeautifulSoup(r2.content, 'html.parser')

table = h2.find('tbody')
team_links = []
_ = [team_links.append(i) if str(i)[:15] == '<a href="/teams' else None for i in table.findAll('a')]

In [ ]:
teams = []
players = []

for link in team_links:
    short, name = link.attrs['href'].split('/')[-2], link.text

    team, player = get_data(short, name)
    teams.append(team)
    players.append(player)
    print(f'Finished {name}')
    time.sleep(5)

In [105]:
pd.concat(teams).reset_index(drop=True).to_csv('teams.csv', index=False)

In [101]:
pd.concat(players).reset_index(drop=True).to_csv('players.csv', index=False)

## Reloading the Data

In [1]:
import pandas as pd

In [129]:
teams = pd.read_csv('teams.csv')

In [73]:
# 'code': 'code',
# 'team': 'team',
# 'Coach': 'Coach',
# 'Offensive Coordinator': 'Offensive Coordinator',
# 'Defensive Coordinator': 'Defensive Coordinator',
# 'Stadium': 'Stadium',

owner_cols = {
    'Principal Owner': 'Owner',
    'President': 'Owner',
    'Chair': 'Owner',
    'Co-Chairman': 'Owner',
    'Chairman/Executive Vice President/Co-Owner': 'Owner',
    'Owner/President': 'Owner',
    'Owner/CEO': 'Owner',
    'Majority Owner': 'Owner',
    'Secretary of the Board of Directors': 'Owner',
    'Principal Owner/President': 'Owner',
    'Owner/President/General Manager': 'Owner',
    'President/CEO': 'Owner',
    'Principal Owner/CEO': 'Owner',
    'Chairman/CEO': 'Owner',
    'Owner': 'Owner',
    'Controlling Owner/Chairman': 'Owner',
    'Owner/Chairman': 'Owner',
    'Chairman/Managing General Partner': 'Owner',
    'Principal Owner/Chairman': 'Owner',
    'Owner/Chairman of the Board': 'Owner',
}

gm_cols = {
    'General Manager': 'General Manager',
    'General Manager/Executive VP': 'General Manager',
    'General Manager/Executive VP of Football Operations': 'General Manager',
    'Director of Scouting/de facto GM': 'General Manager',
    'Executive VP/General Manager': 'General Manager',
    'Exec. VP of Football Ops/General Manager': 'General Manager',
    'Director of Player Personnel': 'General Manager',
    'Owner/President/General Manager': 'General Manager',
}

In [74]:
team_data = []
for i in range(32):
    s = teams.iloc[i]

    t = s[['code', 'team', 'Coach', 'Offensive Coordinator', 'Defensive Coordinator', 'Stadium']]

    owner = None
    for c in list(owner_cols.keys()):
        if str(s[c]) != 'nan':
            owner = s[c]
    if owner is None:
        print(f'No owner for team {s.name}')
    t['Owner'] = owner
        
    gm = None
    for c in list(gm_cols.keys()):
        if str(s[c]) != 'nan':
            gm = s[c]
    if gm is None:
        print(f'No gm for team {s.name}')

    t['General Manager'] = gm

    team_data.append(t)

In [89]:
teams = pd.DataFrame(team_data)

In [90]:
teams.loc[teams['code'] == 'sfo', 'Offensive Coordinator'] = 'Chris Foerster'
teams.loc[teams['code'] == 'sfo', 'Stadium'] = "Levi's Stadium"
teams.loc[teams['code'].isna(), 'code'] = 'was'
teams.loc[teams['team'].isna(), 'team'] = 'Washington Commanders'

In [ ]:
teams.rename(columns={'team':'name', 'Coach':'coach', 'Offensive Coordinator':'off_coord', 'Defensive Coordinator':'def_coord', 'Stadium':'stadium', 'Owner':'owner', 'General Manager':'gm'}, inplace=True)

In [145]:
teams.to_csv('teams.csv', index=False)

In [130]:
teams.head()

,code,team,Coach,Offensive Coordinator,Defensive Coordinator,Stadium,Owner,General Manager
0,crd,Arizona Cardinals,Jonathan Gannon,Drew Petzing,Nick Rallis,State Farm Stadium,Michael Bidwill,Monti Ossenfort
1,atl,Atlanta Falcons,Raheem Morris,Zac Robinson,Jimmy Lake,Mercedes-Benz Stadium,Arthur Blank,Terry Fontenot
2,rav,Baltimore Ravens,John Harbaugh,Todd Monken,Zach Orr,M&T Bank Stadium,Steve Bisciotti,Eric DeCosta
3,buf,Buffalo Bills,Sean McDermott,Joe Brady,Bobby Babich,Highmark Stadium,Terry Pegula,Brandon Beane
4,car,Carolina Panthers,Dave Canales,Brad Idzik,Ejiro Evero,Bank of America Stadium,David Tepper,Dan Morgan


In [164]:
players = pd.read_csv('players.csv')

In [51]:
players.yrs = players.yrs.replace('Rook', '0')
# players.reset_index(inplace=True, names='id')
players['status'] = players.player.str.split(' \(').str[1].str[:-1].fillna('ACT')
players.player = players.player.str.split(' \(').str[0]

players = players.fillna({'draft': ' /  /  / ', 'ht': '0-0'})

for i in range(len(players)):
    draft = players.iloc[i].draft.split(' / ')
    players.loc[players.index == i, 'draft_team'] = draft[0]
    players.loc[players.index == i, 'draft_year'] = draft[3]
    players.loc[players.index == i, 'round'] = draft[1][:-2]
    players.loc[players.index == i, 'pick'] = draft[2][:-7]

players = players.replace('', 'NULL').drop(columns=['draft']).fillna('NULL')
players.ht = (players.ht.str.split('-').str[0].astype(int) * 12 + 
              players.ht.str.split('-').str[1].astype(int)).replace(0, 'NULL')

In [55]:
players = players[['code', 'team', 'player', 'num', 'pos', 'age', 'yrs', 'g', 'gs', 'status', 'ht', 
         'wt', 'birth', 'college', 'draft_team', 'draft_year', 'round', 
         'pick']].rename(columns={'g':'games_played', 'gs':'games_started'})

players.to_csv('players.csv', index=False)

In [176]:
players = pd.read_csv('players.csv')

In [177]:
players = players.fillna('NULL')

In [178]:
players.num = players.num.astype(str).str.replace('.0', '', regex=False)
players.age = players.age.astype(str).str.replace('.0', '', regex=False)
players.ht = players.ht.astype(str).str.replace('.0', '', regex=False)
players['round'] = players['round'].astype(str).str.replace('.0', '', regex=False)
players.pick = players.pick.astype(str).str.replace('.0', '', regex=False)

In [179]:
players.rename(columns={'yrs':'years', 'ht':'height', 'wt':'weight'}, inplace=True)

In [211]:
players.draft_year = players.draft_year.str[:4]

In [146]:
teams = pd.read_csv('teams.csv')
players = pd.read_csv('players.csv')

In [212]:
players.head()

,code,team,player,num,pos,age,years,games_played,games_started,status,height,weight,birth,college,draft_team,draft_year,round,pick
0,crd,Arizona Cardinals,Isaiah Adams,74,OL,24,0,4,0,ACT,NULL,316,7/21/2000,"Wilfrid Laurier,Garden City (KS),Illinois",Arizona Cardinals,2024,3,71
1,crd,Arizona Cardinals,Budda Baker,3,S,28,7,4,4,ACT,70,195,1/10/1996,Washington,Arizona Cardinals,2017,2,36
2,crd,Arizona Cardinals,Krys Barnes,51,LB,26,4,4,0,ACT,74,229,4/2/1998,UCLA,NULL,NULL,NULL,NULL
3,crd,Arizona Cardinals,Kelvin Beachum,68,OL,35,12,3,2,ACT,75,308,6/8/1989,SMU,Pittsburgh Steelers,2012,7,248
4,crd,Arizona Cardinals,Trey Benson,33,RB,21,0,4,1,ACT,73,223,7/21/2003,"Oregon,Florida St.",Arizona Cardinals,2024,3,66


In [218]:
types = ['char(3) not null', 'varchar(45) not null', 'varchar(45) not null', 
         'varchar(45) not null', 'varchar(45) not null', 'varchar(45) not null', 
         'varchar(45) not null', 'varchar(45) not null']

script = 'drop table if exists team;\n'
script += 'create table team (\n\t`id` int not null auto_increment primary key,\n'
for col, t in zip(teams.columns, types):
    script += f'\t`{col}` {t},\n'
script = script[:-2] + '\n);\n'

script += f"\ninsert into team({', '.join(teams.columns)}) values\n"
for i in range(len(teams)):
    team = teams.iloc[i]
    
    line = '\t('
    for col, t in zip(team.values, types):
        line += f'"{col}"'
        line += ', '
    script += line[:-2] + '),\n'
    
script = script[:-2] + ';\n'

In [219]:
types = ['char(3) not null', 'varchar(45) not null', 'varchar(45) not null', 'int', 
         'varchar(5)', 'int', 'int', 'int', 'int', 'varchar(5)', 'int', 'int', 
         'varchar(10)', 'varchar(100)', 'varchar(45)', 'int', 'int', 'int']

script += '\ndrop table if exists player;\n'
script += 'create table player (\n\t`id` int not null auto_increment primary key,\n'
for col, t in zip(players.columns, types):
    script += f'\t`{col}` {t},\n'
script = script[:-2] + '\n);\n'

script += f'\ninsert into player({", ".join(players.columns)}) values\n'
for i in range(len(players)):
    player = players.iloc[i]
    
    line = '\t('
    for col, t in zip(player.values, types):
        if t == 'int':
            line += str(col)
        else:
            if col == 'NULL':
                line += 'NULL'
            else:
                line += f'"{col}"'
        line += ', '
    script += line[:-2] + '),\n'
    
script = script[:-2] + ';\n'

In [220]:
with open('load_data.sql', 'w', encoding='utf-8') as f:
    f.write(script)

In [1]:
with open('load_data.sql', 'r', encoding='utf-8') as f:
    script = f.read()

In [8]:
script = script.replace("'", "''").replace('"', "'").replace('`', '').replace('int not null auto_increment', 'serial')

In [9]:
with open('load_post.sql', 'w', encoding='utf-8') as f:
    f.write(script)